In [1]:
import os
import numpy as np 
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.image  import load_img, img_to_array
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Lambda, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
import zipfile

In [8]:
train_path_zip = '/content/drive/My Drive/ML/siamese_train.zip'
test_path_zip =  '/content/drive/My Drive/ML/siamese_test.zip'

In [10]:
def extract(path):

  local_zip = path

  zip_ref = zipfile.ZipFile(local_zip, 'r')

  zip_ref.extractall('/tmp')
  zip_ref.close()

In [11]:
extract(train_path_zip)
extract(test_path_zip)

In [12]:
train_path = '/tmp/train/'
test_path = '/tmp/test/'

In [13]:
def folder_to_arr(path):
  X = []
  for alphabet in os.listdir(path):
    alphabet_path = path + alphabet + '/'
    

    for char in os.listdir(alphabet_path):
      char_path = alphabet_path + char + '/'
      char_list = []

      for file in os.listdir(char_path):
        file_path = char_path + file
        img = load_img(file_path)
        img_array = img_to_array(img)
        char_list.append(img_array[:,:,0])

      X.append(char_list)		


  X  = np.array(X)
  return X

In [14]:
X_train = folder_to_arr(train_path)
X_test = folder_to_arr(test_path)

In [15]:
def get_batch(batch_size, arr):
    m,nr,h,w = arr.shape


    pairs = [np.zeros((batch_size, h, w)) for i in range(2)]
    targets = [np.zeros((batch_size,))]

    targets[0][:batch_size//2] = 1

    randomlist1 = random.sample(range(m), batch_size)
    randomlist4 = random.sample(range(m), batch_size)

    for i in range(batch_size):
        idx1 = random.sample(range(nr), 1)
        idx2 = random.sample(range(nr),1)
        pairs[0][i,:,:] = arr[randomlist1[i], idx1,:,:]

        if i < batch_size//2:
            pairs[1][i,:,:] = arr[randomlist1[i], idx2,:,:]
        else:
            pairs[1][i,:,:] = arr[randomlist4[i], idx2, :,:]
    return pairs, targets

In [16]:
def initialize_weights(shape, dtype):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

def initialize_bias(shape, dtype):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [17]:
def get_siamese_model(input_shape):

  left_input = Input(input_shape)
  right_input = Input(input_shape)
  
  model = Sequential()
  model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))

  model.add(MaxPooling2D())

  model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))

  model.add(MaxPooling2D())

  model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))

  model.add(MaxPooling2D())

  model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))

  model.add(Flatten())

  model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))

  encoded_l = model(left_input)
  encoded_r = model(right_input)

  L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
  L1_distance  = L1_layer([encoded_l, encoded_r])


  prediction = Dense(1, activation = 'sigmoid', bias_initializer = initialize_bias)(L1_distance)

  siamese_net = Model(inputs = [left_input, right_input], outputs = prediction)

  return siamese_net

In [18]:
model = get_siamese_model((105, 105, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4096)         0           sequential[1][0]             

In [19]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [20]:
for i in range(20000):
    inputs, targets = get_batch(32, X_train)
    loss  =  model.train_on_batch(inputs, targets)
    if i%200 == 0:
      print('After '+str(i)+' iterations the cost is '+ str(loss) )

In [21]:
test_batch, test_labels = get_batch(20, X_test)
preds = model.predict(test_batch)

print(preds)
print(test_labels)


[[9.0311432e-01]
 [9.7503018e-01]
 [9.8054928e-01]
 [8.2030427e-01]
 [5.7851744e-01]
 [3.9635342e-01]
 [8.9999211e-01]
 [9.5573580e-01]
 [9.7042727e-01]
 [8.4849107e-01]
 [1.9684363e-02]
 [6.2516336e-03]
 [4.8260537e-01]
 [3.7863767e-03]
 [5.8793314e-05]
 [2.3412744e-05]
 [3.0391885e-04]
 [2.9982508e-05]
 [1.4938248e-03]
 [2.6558533e-01]]
[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])]


In [24]:
def create_nway_task(n_way):
  m,nr,h,w = X_test.shape

  pairs = [np.zeros((n_way, h, w)) for i in range(2)]
  randomlist = random.sample(range(m), 1)
  randomlist1 = random.sample(range(nr), 1)
  pairs[0][:,:,:] = X_test[randomlist, randomlist1,:,:]
  nway_list = random.sample(range(m), n_way)
  
  for a in range(n_way):
    idx = random.sample(range(nr), 1)
    pairs[1][a,:,:] = X_test[nway_list[a], idx,:,:]
  
  target = random.sample(range(n_way), 1)
  idx = random.sample(range(nr), 1)
  pairs[1][target, :, :] = X_test[randomlist, idx,:,:]

  return pairs, target[0]

In [44]:
targets = []
predictions = []
n_way = 5
iters = 250
for j in range(iters):
  inputs, target = create_nway_task(n_way)
  preds = model.predict(inputs)
  prediction = np.argmax(preds)
  targets.append(target)
  predictions.append(prediction)

acc = np.array(targets)==np.array(predictions) 
print('Overall accuracy for ' +str(n_way) +' way one_shot learning after ' + str(iters) + ' iterations is ' + str(np.sum(acc)*100/len(acc)) +'%.')

Overall accuracy for 5 way one_shot learning after 250 iterations is 91.2%.
